In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import h5py

import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or '3' to suppress most of the logs
tf.get_logger().setLevel('WARNING')  # Adjust logging level

2023-12-20 23:17:13.849176: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-20 23:17:13.872725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 23:17:13.872746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 23:17:13.873445: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 23:17:13.877419: I tensorflow/core/platform/cpu_feature_guar

In [114]:
X = np.load('images64.npy')
y = np.load('styles64.npy')
# X and y are now numpy arrays containing your data

In [115]:
np.unique(y)

array(['Classic', 'Contemporary', 'Country', 'Minimalism', 'Modern',
       'Unique', 'Urban'], dtype='<U12')

In [116]:
from sklearn.calibration import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)



from tensorflow.keras.utils import to_categorical

# Assuming y contains integer labels
y = to_categorical(y, num_classes=7)

In [95]:
from sklearn.model_selection import train_test_split
 

# Now proceed with the train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

In [96]:
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(6970, 64, 64, 3) (871, 64, 64, 3) (872, 64, 64, 3)
(6970, 7) (871, 7) (872, 7)


In [97]:
X_train.shape[1:]

(64, 64, 3)

In [98]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Dropout, Conv2D, AvgPool2D, Flatten, LeakyReLU

In [99]:
model = Sequential([
    Input(X_train.shape[1:]),
    
    Conv2D(filters=32, kernel_size=(4,4), strides=(1,1), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    AvgPool2D(pool_size=(4,4)),
    
    Conv2D(filters=16, kernel_size=(4,4), strides=(1,1), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    AvgPool2D(pool_size=(4,4)),
    
    Conv2D(filters=8, kernel_size=(4,4), strides=(1,1), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    AvgPool2D(pool_size=(4,4)),
    
    Flatten(),
    
    Dense(128),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.2),
    
    Dense(7),
    Activation('softmax')
])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 64, 64, 32)        1568      
                                                                 
 batch_normalization_16 (Ba  (None, 64, 64, 32)        128       
 tchNormalization)                                               
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 64, 64, 32)        0         
                                                                 
 average_pooling2d_12 (Aver  (None, 16, 16, 32)        0         
 agePooling2D)                                                   
                                                                 
 conv2d_13 (Conv2D)          (None, 16, 16, 16)        8208      
                                                                 
 batch_normalization_17 (Ba  (None, 16, 16, 16)       

In [100]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               verbose=1,
                               restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=5,
                              verbose=1,
                              min_lr=0.001)


In [101]:
# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print('start fitting')
history = model.fit(
    X_train, y_train,
    epochs=10000000,
    batch_size=32,
    validation_data=(X_val,y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1,
)

print('fitting done')

Num GPUs Available:  1
start fitting
Epoch 1/10000000
218/218 [==============================] - 2s 4ms/step - loss: 1.1221 - accuracy: 0.6680 - val_loss: 1.3352 - val_accuracy: 0.7061 - lr: 0.0010
Epoch 2/10000000
218/218 [==============================] - 1s 3ms/step - loss: 0.9581 - accuracy: 0.7036 - val_loss: 0.9719 - val_accuracy: 0.7049 - lr: 0.0010
Epoch 3/10000000
218/218 [==============================] - 1s 3ms/step - loss: 0.9170 - accuracy: 0.7164 - val_loss: 0.9805 - val_accuracy: 0.7049 - lr: 0.0010
Epoch 4/10000000
218/218 [==============================] - 1s 3ms/step - loss: 0.8811 - accuracy: 0.7209 - val_loss: 1.2178 - val_accuracy: 0.5821 - lr: 0.0010
Epoch 5/10000000
218/218 [==============================] - 1s 3ms/step - loss: 0.8485 - accuracy: 0.7314 - val_loss: 1.0074 - val_accuracy: 0.7095 - lr: 0.0010
Epoch 6/10000000
218/218 [==============================] - 1s 3ms/step - loss: 0.8294 - accuracy: 0.7406 - val_loss: 1.1621 - val_accuracy: 0.6958 - lr: 0.00

In [104]:
y_pred = model.predict(X_test)
print(y_pred[0])

import numpy as np

y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels[:10]



28/28 [==============================] - 0s 993us/step
[1.3711634e-03 7.0549240e-03 7.3964198e-05 1.2461369e-05 9.9145377e-01
 2.9698979e-08 3.3737851e-05]


array([4, 4, 2, 4, 4, 0, 4, 4, 4, 4])

In [105]:
if y_test.ndim > 1:  # Check if y_test is one-hot encoded
    y_test_int_labels = np.argmax(y_test, axis=1)
else:
    y_test_int_labels = y_test  # y_test is already in integer label format

# Now calculate the accuracy
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Accuracy:", accuracy_score(y_test_int_labels, y_pred_labels))
print("Confusion Matrix:\n", confusion_matrix(y_test_int_labels, y_pred_labels))
print("Classification Report:\n", classification_report(y_test_int_labels, y_pred_labels))


Accuracy: 0.7958715596330275
Confusion Matrix:
 [[ 49   2   1   3  55   0   0]
 [  2  20   3   0  39   0   0]
 [  5   3  22   0  14   0   0]
 [  1   2   1   5   5   0   0]
 [ 18  12   4   1 579   0   1]
 [  1   0   0   0   1   3   0]
 [  2   0   0   0   2   0  16]]
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.45      0.52       110
           1       0.51      0.31      0.39        64
           2       0.71      0.50      0.59        44
           3       0.56      0.36      0.43        14
           4       0.83      0.94      0.88       615
           5       1.00      0.60      0.75         5
           6       0.94      0.80      0.86        20

    accuracy                           0.80       872
   macro avg       0.74      0.57      0.63       872
weighted avg       0.78      0.80      0.78       872



In [2]:
# Load one image to get the input shape
with h5py.File('image.h5', 'r') as h5file:
    one_file = h5file['images'][0:1]  # Load the first image
    

In [3]:
one_file.shape

(1, 1024, 1024, 3)

In [4]:
from sklearn.calibration import LabelEncoder

y = np.load('styles.npy', allow_pickle=True)
le = LabelEncoder()
y = le.fit_transform(y)

In [5]:
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6])

In [6]:
y.shape

(8713,)